In [1]:
import numpy as np
from rl_env_sarl import SARLDraftEnv
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from pprint import pprint

/Users/ashish/files/fun_projects/fantasy_football_2024/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


in order to test how robust this is against managers with different ideas of player values, I should have the stochastic choice select a random projection source.


model: PPO_20240827-124619
- pretty good model - best so far - rewarded for first place in starters
- I need to try turning off the KL divergence limit bc it's early stopping a lot which could prevent it from finding new strategies. 
- I also think the temperature on the random choices from other players might have been a bit too high so i will turn it down.

model: logs/PPO_20240827-143658/best_model.zip
- works very well

- based on this article, https://fantasyfootballanalytics.net/2015/07/accuracy-of-fantasy-football-projections-interactive-scatterplot-in-r.html, DEF projections are not very reliable. maybe i should override rec to draft early
    - i also know i'll need backup players of injured players
    
TODO:
- need to rescrape and update projections
- need to estimate conditional variance of each player from previous year and put a variance penalty on reward
- make a method that offers me next best choice 
- in order to understand reliability of projections, need to look at correlation between projections and performance in previous years
- look at whether the non penalized (def/k) algorithm shows worse performance of defense by the opponents
- for draft interface, I need to immediately pull up plots with mean, proj std, and hist std for top few players in each position. also include bye week for bye week based decisions.k


- prevent multiple def or k
- give points for flex past starters
- think about bye weeks - add to plotting


- bots are choosing def and kicker too early. i put in a temp fix, but make it better



In [2]:

# class learning_rate_schedule:
#     def __init__(self, initial_lr=1e-4, final_lr=5e-5):
#         self.initial_lr = initial_lr
#         self.final_lr = final_lr
    
#     def __str__(self):
#         return f"lr_{self.initial_lr}to{self.final_lr}"

#     def __call__(self, progress_remaining):
#         return progress_remaining * (self.initial_lr - self.final_lr) + self.final_lr

# class adaptive_ent_coef:
#     def __init__(self, initial_ent_coef=0.03, final_ent_coef=0.01):
#         self.initial_ent_coef = initial_ent_coef
#         self.final_ent_coef = final_ent_coef
        
#     def __str__(self):
#         return f"ent_{self.initial_ent_coef}to{self.final_ent_coef}"
    
#     def __call__(self, progress_remaining):
#         return progress_remaining * (self.initial_ent_coef - self.final_ent_coef) + self.final_ent_coef
    
def learning_rate_schedule(initial_lr=1e-4, final_lr=5e-5):
    return lambda progress_remaining: progress_remaining * (initial_lr - final_lr) + final_lr

learning_rate_schedule_fn = learning_rate_schedule(initial_lr=1e-4, final_lr=5e-5)


ppo_params = {
    "policy": "MlpPolicy",
    "learning_rate": learning_rate_schedule_fn,  # Adaptive learning rate
    "n_steps": 750,
    "batch_size": 750,
    "n_epochs": 30,
    "gamma": 0.99,
    "gae_lambda": 0.95,
    "clip_range": 0.2,
    "clip_range_vf": None,
    "normalize_advantage": True,
    "ent_coef": .03, # Entropy coefficient for the loss calculation
    "vf_coef": 0.5,
    "max_grad_norm": 0.7,
    "use_sde": False,
    "sde_sample_freq": -1,
    "rollout_buffer_class": None,
    "rollout_buffer_kwargs": None,
    # "target_kl": 0.01,
    # "target_kl": None, # TODO: TRY THIS NEXT TEST BECAUSE KEEP GETTING EARLY STOPPING
    "stats_window_size": 100,
    "policy_kwargs": dict(net_arch=[dict(pi=[256, 256, 128], vf=[256, 256, 128])]),
    "verbose": 1,
    "seed": 69,
    "device": "auto",
    "_init_setup_model": True
}


In [3]:
check_env(SARLDraftEnv(stochastic_temp=.5))

In [4]:
# env = SARLDraftEnv()
# model = DQN(env=env, **dqn_params)
# model.learn(total_timesteps=100000)

In [5]:
import os
import json
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback, CheckpointCallback, EvalCallback
from stable_baselines3.common.logger import configure
from datetime import datetime

notes = """
- starting from model logs/PPO_20240828-000621/best_model.zip
- penalizing >1 DEF and >1 K
- half rewards for RB and WR past starters
- half rewards for first QB past starter
- quarter rewards for TE past starters
^^^^^ everything above this is when things got worse according to simulations
- added rules to stochastic choice for opponents so that they'll pick def and k later
- added penalty for picking K before round 12
- reduced bye week penalty to 1/(17*2) for each non completed week
"""

# Define your environment
env = SARLDraftEnv(stochastic_temp=.5)

# Generate a unique run ID
run_id = datetime.now().strftime("%Y%m%d-%H%M%S")

# Choose significant parameters to include in the log directory name
lr_str = ppo_params['learning_rate'].__name__ if callable(ppo_params['learning_rate']) else ppo_params['learning_rate']
gamma_str = ppo_params['gamma'].__name__ if callable(ppo_params['gamma']) else ppo_params['gamma']
param_str = f"PPO"
# param_str = f"PPO_lr_{lr_str}_gamma_{gamma_str}"
log_dir = f"./logs/{param_str}_{run_id}/"
os.makedirs(log_dir, exist_ok=True)

# Save ppo_params to a JSON file
params_save_path = os.path.join(log_dir, f'ppo_params_{run_id}.json')
with open(params_save_path, 'w') as f:
    # if the param is not json serializable (e.g. function), then write callable name
    json_params = {key: value if not callable(value) else value.__name__ for key, value in ppo_params.items()}
    json_params['stochastic_temp'] = env.stochastic_temp
    json_params['notes'] = notes
    json.dump(json_params, f, indent=2)

# Configure the logger
new_logger = configure(log_dir, ["stdout", "csv", "tensorboard"])

# Define the PPO model and attach the logger
# model = PPO(**ppo_params, env=env, tensorboard_log=log_dir)
# model = PPO.load("logs/PPO_20240827-143658/best_model.zip", env=env) # load model from previous run
# model = PPO.load("logs/PPO_20240829-001139/best_model.zip", env=env) # load model from previous run
# model = PPO.load("logs/PPO_20240829-022850/best_model.zip", env=env) # load model from previous run
model = PPO.load("logs/PPO_20240828-000621/best_model.zip", env=env) # load model from previous run
model.set_logger(new_logger)


# Log ppo_params to TensorBoard/CSV
for key, value in ppo_params.items():
    model.logger.record(f'params/{key}', value)


# Custom callback for monitoring and saving during training
class TrainingMonitorCallback(BaseCallback):
    def __init__(self, log_interval: int, draft_save_interval: int, log_dir: str, run_id: str, verbose=1):
        super(TrainingMonitorCallback, self).__init__(verbose)
        self.log_interval = log_interval
        self.draft_save_interval = draft_save_interval
        self.log_dir = log_dir
        self.run_id = run_id
        self.best_mean_reward = -float("inf")
        self.rewards_history = []
        self.losses_history = []
        self.episode_counter = 0
        self.draft_history = []  # List to store draft dicts

        self.model_save_path = os.path.join(log_dir, f'best_model_{run_id}')
        self.rewards_losses_path = os.path.join(log_dir, f'rewards_losses_{run_id}.json')
        self.drafts_save_path = os.path.join(log_dir, f'draft_history_{run_id}.json')

    def _init_callback(self) -> None:
        os.makedirs(self.log_dir, exist_ok=True)

    def _on_step(self) -> bool:

        # Check if the episode has terminated
        if self.locals['dones'][0]:
            
            # Log losses if they are available (PPO does not use TD errors)
            if 'loss' in self.locals:
                current_loss = self.locals['loss']
                self.losses_history.append(current_loss)
                
            # Log rewards
            current_reward = self.locals['rewards']
            self.rewards_history.append(current_reward)
            
            # Periodic logging and saving
            if self.episode_counter % self.log_interval == 0:
                mean_reward = np.mean(self.rewards_history[-self.log_interval:])
                self.logger.record('mean_reward', mean_reward)

                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    self.model.save(self.model_save_path)
                    if self.verbose > 0:
                        print(f"New best mean reward: {self.best_mean_reward}. Model saved to {self.model_save_path}")
                
                # Convert rewards and losses to lists before saving
                rewards_list = [float(reward.item() if isinstance(reward, np.ndarray) else reward) for reward in self.rewards_history]
                losses_list = [float(loss.item() if isinstance(loss, np.ndarray) else loss) for loss in self.losses_history]

                # Save rewards and losses to a file
                with open(self.rewards_losses_path, 'w') as f:
                    json.dump({'rewards': rewards_list, 'losses': losses_list}, f)
            
            if self.episode_counter % self.draft_save_interval == 0:
                draft = self.locals['infos'][0]['draft']
                self.draft_history.append(draft)
                with open(self.drafts_save_path, 'w') as f:
                    json.dump(self.draft_history, f, indent=4)
                
            self.episode_counter += 1

        return True

# Initialize and use the custom callback
monitor_callback = TrainingMonitorCallback(log_interval=50, # each 50 episodes, save rewards, losses, best model
                                           draft_save_interval=50,  # Every 250 episodes save draft
                                           log_dir=log_dir, 
                                           run_id=run_id)

# Create a checkpoint callback to save the model periodically
checkpoint_callback = CheckpointCallback(save_freq=5000, save_path=log_dir, name_prefix='ppo_model')

# Gets called when there is new best model
eval_callback = EvalCallback(env, best_model_save_path=log_dir, log_path=log_dir, eval_freq=5000, deterministic=True, render=False)

# Combine all callbacks
callback = [checkpoint_callback, eval_callback, monitor_callback]

# Train the model
model.learn(total_timesteps=2e6, callback=callback)

Logging to ./logs/PPO_20240830-021915/
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/ashish/files/fun_projects/fantasy_football_2024/venv/lib/python3.9/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


New best mean reward: -0.5151620507240295. Model saved to ./logs/PPO_20240830-021915/best_model_20240830-021915
-------------------------------------------------------------------
| mean_reward              | -0.51516205                          |
| params/                  |                                      |
|    _init_setup_model     | True                                 |
|    batch_size            | 750                                  |
|    clip_range            | 0.2                                  |
|    clip_range_vf         | None                                 |
|    device                | auto                                 |
|    ent_coef              | 0.03                                 |
|    gae_lambda            | 0.95                                 |
|    gamma                 | 0.99                                 |
|    learning_rate         | <function learning_rate_schedule.... |
|    max_grad_norm         | 0.7                                  |
|   

/Users/ashish/files/fun_projects/fantasy_football_2024/venv/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=5000, episode_reward=-0.55 +/- 0.00
Episode length: 15.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 15          |
|    mean_reward          | -0.547      |
| mean_reward             | -0.44621027 |
| time/                   |             |
|    total_timesteps      | 5000        |
| train/                  |             |
|    approx_kl            | 0.016182184 |
|    clip_fraction        | 0.115       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.69       |
|    explained_variance   | -0.283      |
|    learning_rate        | 9.99e-05    |
|    loss                 | -0.0727     |
|    n_updates            | 180         |
|    policy_gradient_loss | -0.0154     |
|    value_loss           | 0.0146      |
-----------------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 15.

...

/Users/ashish/files/fun_projects/fantasy_football_2024/venv/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=410000, episode_reward=1.58 +/- 0.03
Episode length: 15.00 +/- 0.00
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 15         |
|    mean_reward          | 1.58       |
| mean_reward             | 1.408915   |
| time/                   |            |
|    total_timesteps      | 410000     |
| train/                  |            |
|    approx_kl            | 0.01507441 |
|    clip_fraction        | 0.106      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.983     |
|    explained_variance   | 0.0136     |
|    learning_rate        | 8.98e-05   |
|    loss                 | -0.0199    |
|    n_updates            | 16380      |
|    policy_gradient_loss | -0.0249    |
|    value_loss           | 0.11       |
----------------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 15.1     |
|    ep_rew_mean     | 1.44     

/Users/ashish/files/fun_projects/fantasy_football_2024/venv/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=415000, episode_reward=1.60 +/- 0.03
Episode length: 15.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 15          |
|    mean_reward          | 1.6         |
| time/                   |             |
|    total_timesteps      | 415000      |
| train/                  |             |
|    approx_kl            | 0.012866376 |
|    clip_fraction        | 0.138       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.01       |
|    explained_variance   | 0.0747      |
|    learning_rate        | 8.96e-05    |
|    loss                 | -0.0499     |
|    n_updates            | 16590       |
|    policy_gradient_loss | -0.0257     |
|    value_loss           | 0.0477      |
-----------------------------------------
----------------------------------
| mean_reward        | 1.4070166 |
| rollout/           |           |
|    ep_len_mean     | 15.1      |
|    ep_rew_mean 

KeyboardInterrupt: 